**Importing and Cleaning**

This notebook serves as introduction to the Kaggle Competition for Text Normalization. Whose data is available here: https://www.kaggle.com/c/text-normalization-challenge-english-language 

I will be using various packages such as numpy for computations, pandas for data frames, and MatPlotLib for visualizations. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [23]:
trainFile = "C:/Users/Kycool13/Documents/GitHub/Kaggle_Text_Normalization/en_train.csv/en_train.csv"
train = pd.read_csv(trainFile)
train.shape

(9918441, 5)

In [19]:
train.head()

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,Brillantaisia,Brillantaisia
1,0,1,PLAIN,is,is
2,0,2,PLAIN,a,a
3,0,3,PLAIN,genus,genus
4,0,4,PLAIN,of,of


In [20]:
testFile= "C:/Users/Kycool13/Documents/GitHub/Kaggle_Text_Normalization/en_test.csv/en_test.csv"
test = pd.read_csv(testFile)
test.shape

(1088564, 3)

In [21]:
test.head()

,sentence_id,token_id,before
0,0,0,Another
1,0,1,religious
2,0,2,family
3,0,3,is
4,0,4,of
